In [1]:
# Import ignore warnings

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing libraries

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
# Prep data

data = Path('../Resources/train_data_feb_day.csv')
df = pd.read_csv(data)
df.head()

,Actual.Stop.Station,New.Trip.Name,Trip_date,Actual.Station.Dprt.Time,Segment.Direction,Trip.Name,Service.Line,Orig..Station,Dest..Station,Leading.Set.Type,Actual.Station.Dprt.Time.Band,Occupancy_Status,Occupancy_Range
0,Aberdeen,V603-JOINED,1-Feb-19,11:48.0,Down,V603,Hunter Valley,Newcastle Interchange,Scone,NaN,05:00-05:14,Many Seats Available,Low: 0-399
1,Aberdeen,V618-JOINED,1-Feb-19,16:12.0,Up,V618,Hunter Valley,Scone,Newcastle Interchange,NaN,06:15-06:29,Many Seats Available,Low: 0-399
2,Aberdeen,V659-JOINED,1-Feb-19,14:48.0,Down,V659,Hunter Valley,Newcastle Interchange,Scone,NaN,18:00-18:14,Many Seats Available,Low: 0-399
3,Aberdeen,V676-JOINED,1-Feb-19,15:12.0,Up,V676,Hunter Valley,Scone,Newcastle Interchange,NaN,19:15-19:29,Many Seats Available,Low: 0-399
4,Aberdeen,V679-JOINED,1-Feb-19,44:48.0,Down,V679,Hunter Valley,Newcastle Interchange,Scone,NaN,19:30-19:44,Many Seats Available,Low: 0-399


In [4]:
# Fill NaNs

df.fillna("", inplace=True)

In [5]:
# Create our features

df_features = df.drop(columns="Occupancy_Status")
X = pd.get_dummies(df_features)

# Create our target

y = df.Occupancy_Status

In [6]:
X.describe()

,Actual.Stop.Station_Aberdeen,Actual.Stop.Station_Adamstown,Actual.Stop.Station_Albion Park,Actual.Stop.Station_Allawah,Actual.Stop.Station_Arncliffe,Actual.Stop.Station_Artarmon,Actual.Stop.Station_Ashfield,Actual.Stop.Station_Asquith,Actual.Stop.Station_Auburn,Actual.Stop.Station_Austinmer,...,Actual.Station.Dprt.Time.Band_22:15-22:29,Actual.Station.Dprt.Time.Band_22:30-22:44,Actual.Station.Dprt.Time.Band_22:45-22:59,Actual.Station.Dprt.Time.Band_23:00-23:14,Actual.Station.Dprt.Time.Band_23:15-23:29,Actual.Station.Dprt.Time.Band_23:30-23:44,Actual.Station.Dprt.Time.Band_23:45-23:59,Occupancy_Range_High: Standing Room Only,Occupancy_Range_Low: 0-399,Occupancy_Range_Medium: 400-799
count,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,...,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000,49869.000000
mean,0.000120,0.001043,0.001043,0.003650,0.002827,0.007460,0.006477,0.001785,0.005755,0.000983,...,0.009405,0.009084,0.008803,0.009104,0.008903,0.009084,0.008623,0.020835,0.881991,0.097175
std,0.010968,0.032275,0.032275,0.060302,0.053099,0.086047,0.080219,0.042208,0.075644,0.031331,...,0.096521,0.094876,0.093412,0.094980,0.093937,0.094876,0.092458,0.142832,0.322622,0.296198
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values

y.value_counts()

Many Seats Available    46745
Few Seats Available      2479
Standing Room Only        579
                           66
Name: Occupancy_Status, dtype: int64

In [8]:
# Importing sklearn library
## Create X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(37401, 9584)

In [9]:
# Create the StandardScaler instance

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data

X_train_scale = X_scaler.transform(X_train)
X_test_scale = X_scaler.transform(X_test)

In [12]:
# Importing Logistic regression library

from sklearn.linear_model import LogisticRegression

## Fitting logisitic Regression to training data

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [13]:
# Importing Balanced Accuracy Score library

from sklearn.metrics import balanced_accuracy_score

## Calculated the balanced accuracy score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8065687302130572

In [14]:
# Importing confusion matrix library

from sklearn.metrics import confusion_matrix

## Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[   14,     0,     2,     0],
       [    0,   363,   211,    46],
       [    1,   157, 11529,     0],
       [    0,    32,     0,   113]], dtype=int64)

In [15]:
# Importing Classification Report (imbalanced)

from imblearn.metrics import classification_report_imbalanced

## Print the imbalanced classification report
## Based on y_test and y_pred data

print(classification_report_imbalanced(y_test, y_pred))

                            pre       rec       spe        f1       geo       iba       sup

                           0.93      0.88      1.00      0.90      0.94      0.86        16
 Few Seats Available       0.66      0.59      0.98      0.62      0.76      0.55       620
Many Seats Available       0.98      0.99      0.73      0.98      0.85      0.74     11687
  Standing Room Only       0.71      0.78      1.00      0.74      0.88      0.76       145

         avg / total       0.96      0.96      0.74      0.96      0.84      0.73     12468



In [16]:
# Importing Random Forest Classifier 

from imblearn.ensemble import BalancedRandomForestClassifier

## Resample the training data with the BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [17]:
# Calculated the balanced accuracy score

y_pred_brf = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred_brf)

0.9117791135449644

In [18]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred_brf)

array([[   16,     0,     0,     0],
       [    0,   477,    18,   125],
       [  620,   728, 10339,     0],
       [    0,     1,     0,   144]], dtype=int64)

In [19]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred_brf))

                            pre       rec       spe        f1       geo       iba       sup

                           0.03      1.00      0.95      0.05      0.97      0.95        16
 Few Seats Available       0.40      0.77      0.94      0.52      0.85      0.71       620
Many Seats Available       1.00      0.88      0.98      0.94      0.93      0.86     11687
  Standing Room Only       0.54      0.99      0.99      0.70      0.99      0.98       145

         avg / total       0.96      0.88      0.98      0.91      0.93      0.85     12468



In [20]:
# List the features sorted in descending order by feature importance
## Then displaed the first 10 feature importances

importances = brf.feature_importances_
importances_sort = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sort[:10]

[(0.06715254554557351, 'Occupancy_Range_Low: 0-399'),
 (0.060710385656288106, 'Occupancy_Range_High: Standing Room Only'),
 (0.04714140371691789, 'Occupancy_Range_Medium: 400-799'),
 (0.04449594602930465, 'Leading.Set.Type_'),
 (0.031046608674194762, 'New.Trip.Name_67-A-JOINED'),
 (0.015514578659323165, 'Trip.Name_67-B'),
 (0.012516800029926344, 'Leading.Set.Type_A'),
 (0.010298150100681103, 'Leading.Set.Type_T'),
 (0.00979511973364624, 'Trip.Name_67-A'),
 (0.009780556551451375, 'Orig..Station_Leppington')]